# DATA GENERATION USING ChatGPT API

Install or update the OpenAI Python library first

In [2]:
# update or install the necessary libraries
# !pip install --upgrade openai
# !pip install --upgrade python-dotenv

In [2]:
import csv
import openai
import os
import IPython
from dotenv import load_dotenv
load_dotenv()


## SYNTHETIC DATA GENERATION - USING PROFILES FROM .csv ##

In [21]:
###### USE PROFILES FROM .CSV ######


### USER DEFINED VARIABLES ###
##########################################
openai.api_key = 'xxx'

cat_instrut = 'waarin het functioneren'
cat_instruct2 = None
icf_category = 'd840-d859'
icf_level = 3
icf_def = 'Bijna volledig kunnen werken of studeren, LICHTE beperking/ participatieprobleem (gering, laag)'
persona = f"2_{icf_category}_{icf_level}"
#persona = '2_b140_2'# NAME OF FILE - medical professional position - dietitian, gp, nurse, pediatrician, physical_therapist, physician
num_notes = 15

persona_file = f'../data/data_generation/persona/persona{persona}.csv'
output_file = f'../data//data_generation/data_csv/data_{icf_category}_{icf_level}_{persona}.csv'

### DEFINE MODEL TO USE ###
MODEL = "gpt-3.5-turbo"
##########################################


# OPEN THE PERSONA PROFILE .CSV 
with open(persona_file, 'r') as infile:
    persona_reader = csv.reader(infile, delimiter = ';')
    for row in persona_reader:
        gender, age_group, med_personnel, illness = row

        # SET VALUE TO EACH VARIABLE
        categorie = icf_category
        niveau = icf_level
        med_personeel = med_personnel
        geslacht = gender
        leeftijdsgroep = age_group
        ziekte = illness
        cat_subject = cat_instrut
        
        # PROMPT TO MODEL
        system_content = f"Jij bent een {med_personeel}. Het geslacht, de leeftijdsgroep en de ziekte van uw patiënten worden vermeld in Context. Je schrijft de door ziekte veroorzaakte aandoeningen van patiënten op in een beknopte stijl zonder uitwerking."

        user_content =f''' 
        """
        ### Instruction ###
        Schrijf alstublieft {num_notes} medische notities die {cat_subject} van de patiënt documenteren voor de ICF-categorie en het niveau gegeven in Context, vermijd het gebruik van de leeftijd van de patiënt. 
        Vermijd het veranderen van het geslacht, de leeftijdsgroep of de ziekte van de patiënt. Voeg voor elke noot een betrouwbaarheidsscore tussen 0 - 1 toe.
        Gebruik niet elke keer 'Patiënt'.
        {cat_instruct2}

        ### Context ###
        ICF categorie {categorie} niveau {niveau} - {icf_def}

        ### Output format ###
        Genereer zinnen in csv-formaat, in deze volgorde "nootnummer|noten|confidence_score|{categorie}|{niveau}|{med_personeel}|{geslacht}|{leeftijdsgroep}|{ziekte}", gebruik '|' in plaats van ','. 
        Voeg geen betrouwbaarheidsscore of andere informatie toe aan medische notities.
        """ 
        '''

        # ASK MODEL TO GENERATE OUTPUT
        response = openai.ChatCompletion.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_content},
                {"role": "user", "content": user_content},
            ],
            temperature= 0.95
        )

        # GET OUTPUT FROM MODEL
        out_data = (response)['choices'][0]['message']['content']

        final_data = f"output for: {categorie}, {niveau}, {med_personeel}, {geslacht}, {leeftijdsgroep}, {ziekte}\n{out_data}\n"
        print(out_data)
        print()

        # WRITE OUTPUT TO FILE
        destination = open(output_file , 'a+')
        csvwriter = csv.writer(destination)
        data_reader = csv.reader(final_data.splitlines(), skipinitialspace=True)
        csvwriter.writerows(data_reader)
        destination.flush()
        destination.close()


1|Patiënt kan bijna volledig werken met lichte beperkingen. |0.8|d840-d859|3|ergotherapeut|man/vrouw|18-64|ergotherapie
2|Lichte beperkingen hebben weinig effect op de mogelijkheid van de patiënt om te studeren. |0.9|d840-d859|3|ergotherapeut|man/vrouw|18-64|ergotherapie
3|Patiënt ervaart enige moeilijkheid bij het voltooien van complexe taken. |0.7|d840-d859|3|ergotherapeut|man/vrouw|18-64|ergotherapie
4|Patiënt heeft enige moeite met het opnemen van nieuwe informatie tijdens het leren. |0.6|d840-d859|3|ergotherapeut|man/vrouw|18-64|ergotherapie
5|De patiënt heeft enige moeite met het onthouden van belangrijke informatie. |0.8|d840-d859|3|ergotherapeut|man/vrouw|18-64|ergotherapie
6|Patiënt heeft lichte problemen met het uitvoeren van zijn/haar werk of studieopdrachten binnen de gestelde tijdslimiet. |0.7|d840-d859|3|ergotherapeut|man/vrouw|18-64|ergotherapie
7|Bijna volledig kunnen functioneren op het werk of tijdens het leren ondanks lichte problemen. |0.9|d840-d859|3|ergotherapeut|

## SYNTHETIC DATA GENERATION - USING 1 PROFILE ##

In [33]:
###### GENERATE 1 PROFILE ######
# DUTCH FINAL: USE CHATGPT PROMPT TO GENERATE DATA 


MODEL = "gpt-3.5-turbo"

cat_instruct = 'het eetgedrag'
cat_instruct2 = None
icf_def = 'eten is niet mogelijk; EN/OF volledig afhankelijk van sondevoeding / TPV; VOLLEDIGE stoornis (totaal, ...)'
categorie = 'd550' #icf_category
niveau = 0 #icf_level
med_personeel = 'verpleegster'
geslacht = 'man'
leeftijdsgroep = '65-80'
ziekte = 'covid'

# PROMPT TO MODEL
system_content = f"Jij bent een {med_personeel}. Het geslacht, de leeftijdsgroep en de ziekte van uw patiënten worden vermeld in Context. Je schrijft de door ziekte veroorzaakte aandoeningen van patiënten op in een beknopte stijl zonder uitwerking."

user_content =f''' 
"""
### Instruction ###
Schrijf tien medische notities die {cat_instruct} van de patiënt documenteren voor de ICF-categorie en het niveau gegeven in Context, vermijd het gebruik van de leeftijd van de patiënt. Voeg voor elke noot een betrouwbaarheidsscore tussen 0 - 1 toe. Gebruik niet elke keer 'Patiënt'.
{cat_instruct2}

### Context ###
ICF categorie {categorie} niveau {niveau} - {icf_def}

### Output format ###
Genereer zinnen in csv-formaat, in deze volgorde "nootnummer|noten|confidence_score|{categorie}|{niveau}|{med_personeel}|{geslacht}|{leeftijdsgroep}|{ziekte}", gebruik '|' in plaats van ','. 
""" 
'''

# ### Examples ###
# Geeft wisselend wel/niet antwoord op vragen.
# Was wel wat rommelig. Viel tijdens handelingen in slaap.


response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content},
    ],
    temperature=1.2
)

print(f"output for: {categorie}, {niveau}, {med_personeel}, {geslacht}, {leeftijdsgroep}, {ziekte}\n")
print((response)['choices'][0]['message']['content'])
output_data = (response)['choices'][0]['message']['content']



output for: d550, 0, verpleegster, man, 65-80, covid

1| Sondevoeding is vereist voor voeding | 0.9 | d550 | 0 | verpleegster | man | COVID-19
2| Patent krijgt via buisvoeding | 0.8 | d550 | 0 | verpleegster | vrouw | COVID-19
3| Afhankelijk van Totale Parenterale Voeding (TPV) voor voeding | 0.7 | d550 | 0 | verpleegster | vrouw | COVID-19
4| Er kan geen eten worden ingenomen; moet afhankelijk zijn van voedingstherapie | 0.8 | d550 | 0 | verpleegster | man | COVID-19
5| HIV-patiënt ontvangt sondevoeding. | 0.6 | d550 | 0 | verpleegster | man | HIV
6| De patiënt is afhankelijk van een buisvoeding-afvopt, omdat hij symptomen van anorexia heeft | 0.7 | d550 | 0 | verpleegster | vrouw | Anorexia
7|De patiënt ondergaat chemotherapie en heeft buisvoeding nodig vanwege een geschiedenis van misselijkheid, braken en diarree | 0.6 | d550 | 0 | verpleegster | man | Chemotherapie
8|De patiënt is op het moment niet in staat - om door de ALS- deze kan geen voedsel innemen maar is afhankelijk van so